In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import label_binarize
from sklearn.impute import KNNImputer

In [2]:
df1 = pd.read_csv("/content/Final_Cleaned.csv")
df2 = pd.read_csv("/content/Final_Cleaned.csv")

In [3]:
#Mean
# imputer = SimpleImputer(strategy='mean', missing_values=np.nan)
# df1[['Vit D', 'Calcium']] = imputer.fit_transform(df1[['Vit D', 'Calcium']])

# imputer = SimpleImputer(strategy='mean', missing_values=np.nan)
# df2[['Vit D', 'Calcium']] = imputer.fit_transform(df2[['Vit D', 'Calcium']])


#Median
# imputer = SimpleImputer(strategy='median', missing_values=np.nan)
# df1[['Vit D', 'Calcium']] = imputer.fit_transform(df1[['Vit D', 'Calcium']])

# imputer = SimpleImputer(strategy='median', missing_values=np.nan)
# df2[['Vit D', 'Calcium']] = imputer.fit_transform(df2[['Vit D', 'Calcium']])

In [4]:
#KNN
knn_imputer = KNNImputer(n_neighbors=5)

df1[['Vit D', 'Calcium']] = knn_imputer.fit_transform(df1[['Vit D', 'Calcium']])

df2[['Vit D', 'Calcium']] = knn_imputer.fit_transform(df2[['Vit D', 'Calcium']])

In [5]:
df1['Diagnosis'] = df1['Diagnosis'].replace({'Porosis': "1", 'Penia': "1", 'Normal': "0"})

categorical_columns = ['Gender', 'DM', 'HTN', 'Menopause']
numerical_columns = ['Age', 'BMD', 'T Score', 'Vit D', 'Calcium']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='mean'), numerical_columns),
        ('cat', OneHotEncoder(), categorical_columns)
    ])

In [6]:
X1 = df1.drop(columns=['Diagnosis'])
y1 = df1['Diagnosis']

clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', LogisticRegression())])


clf.fit(X1, y1)

y_pred1 = clf.predict(X1)

accuracy = accuracy_score(y1, y_pred1)
print("Accuracy:", accuracy)

Accuracy: 0.8581907090464548


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [7]:
df_P= df2[df2['Diagnosis'].isin(["Porosis", "Penia"])]
categorical_columns = ['Gender', 'DM', 'HTN', 'Menopause']
numerical_columns = ['Age', 'BMD', 'T Score', 'Vit D', 'Calcium']

preprocessor2 = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='mean'), numerical_columns),
        ('cat', OneHotEncoder(), categorical_columns)
    ])

In [8]:
X2 = df_P.drop(columns=['Diagnosis'])
y2 = df_P['Diagnosis']


clf2 = Pipeline(steps=[('preprocessor', preprocessor2),
                      ('classifier', LogisticRegression())])

clf2.fit(X2, y2)

y_pred_ones_indices = np.where(y_pred1 == "1")[0]
X_pred_ones = X1.iloc[y_pred_ones_indices]
y_pred2 = clf2.predict(X_pred_ones)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [9]:
y_actual = df2.iloc[y_pred_ones_indices]
y_actual_diagnosis=y_actual["Diagnosis"]


accuracy_2 = accuracy_score(y_actual_diagnosis, y_pred2)
print("Accuracy for distinguishing between Porosis and Penia:", accuracy_2)

Accuracy for distinguishing between Porosis and Penia: 0.6648648648648648


In [10]:
y_prob = clf2.predict_proba(X_pred_ones)[:, 1]

y_actual_diagnosis_bin = (y_actual_diagnosis == "Porosis").astype(int)

# Calculate the AUC
auc = roc_auc_score(y_actual_diagnosis_bin, y_prob)

print(f"AUC for the logistic regression model: {auc:.4f}")

AUC for the logistic regression model: 0.7783
